In [ ]:
import os
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# 데이터 전처리 설정 (224x224로 리사이즈, 데이터 증강)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 전체 데이터를 불러옴 (성충과 유충 폴더 모두 포함)
# data_dir = '/content/drive/MyDrive/제4회 AI교육 해커톤/Sample/image_data/data'
data_dir = 'input image data path'
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

## EfficientNet 사용 (training)

In [ ]:
# 데이터셋의 클래스 이름 확인
class_names = full_dataset.classes
print(f"Classes: {class_names}")  # 성충_응애, 성충_정상, 유충_부저병 등 7개 클래스

# 데이터셋 크기 확인
dataset_size = len(full_dataset)
print(f"Dataset size: {dataset_size}")

# 데이터셋을 80% 학습, 20% 검증으로 분리
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 사전 학습된 EfficientNet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.efficientnet_b0(pretrained=True)

# 마지막 출력 레이어 변경 (클래스 수에 맞게) => for transfer learning
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 학습 스케줄러 (에폭 진행에 따라 학습률 조정)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# 모델 학습 함수
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            dataloader = train_loader if phase == 'train' else val_loader

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)
    return model

# 모델 학습
model_ft = train_model(model_ft, criterion, optimizer_ft, scheduler, num_epochs=25)

Classes: ['성충_날개불구바이러스감염증', '성충_응애', '성충_정상', '유충_부저병', '유충_석고병', '유충_응애', '유충_정상']
Dataset size: 329


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You 

Epoch 0/24
----------
train Loss: 1.8831 Acc: 0.2357
val Loss: 1.7826 Acc: 0.4394
Epoch 1/24
----------
train Loss: 1.6881 Acc: 0.5057
val Loss: 1.4611 Acc: 0.7424
Epoch 2/24
----------
train Loss: 1.4232 Acc: 0.6502
val Loss: 1.2203 Acc: 0.7273
Epoch 3/24
----------
train Loss: 1.1974 Acc: 0.7414
val Loss: 1.0208 Acc: 0.7727
Epoch 4/24
----------
train Loss: 1.0434 Acc: 0.7757
val Loss: 0.8720 Acc: 0.7879
Epoch 5/24
----------
train Loss: 0.8860 Acc: 0.8023
val Loss: 0.7766 Acc: 0.8182
Epoch 6/24
----------
train Loss: 0.7815 Acc: 0.8251
val Loss: 0.7076 Acc: 0.8030
Epoch 7/24
----------
train Loss: 0.7293 Acc: 0.8061
val Loss: 0.7155 Acc: 0.8182
Epoch 8/24
----------
train Loss: 0.7462 Acc: 0.8251
val Loss: 0.7167 Acc: 0.8182
Epoch 9/24
----------
train Loss: 0.7376 Acc: 0.8327
val Loss: 0.7046 Acc: 0.8030
Epoch 10/24
----------
train Loss: 0.6702 Acc: 0.8555
val Loss: 0.6734 Acc: 0.8333
Epoch 11/24
----------
train Loss: 0.7254 Acc: 0.8175
val Loss: 0.6840 Acc: 0.8333
Epoch 12/24
--

In [ ]:
torch.save(model_ft.state_dict(), '/content/drive/MyDrive/제4회 AI교육 해커톤/bee_classification_efficientnet.pth')

## ResNet 사용 (training)

In [ ]:
# 데이터셋의 클래스 이름 확인
class_names = full_dataset.classes
print(f"Classes: {class_names}")  # 성충_응애, 성충_정상, 유충_부저병 등 7개 클래스

# 데이터셋 크기 확인
dataset_size = len(full_dataset)
print(f"Dataset size: {dataset_size}")

# 데이터셋을 80% 학습, 20% 검증으로 분리
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 사전 학습된 ResNet18 모델 불러오기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True)

# 마지막 출력 레이어 변경 (총 7개의 클래스에 맞게) => for transfer learning
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 학습 스케줄러 (에폭 진행에 따라 학습률 조정)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# 모델 학습 함수
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            dataloader = train_loader if phase == 'train' else val_loader

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)
    return model

# 모델 학습
model_ft = train_model(model_ft, criterion, optimizer_ft, scheduler, num_epochs=25)

# 모델 저장
torch.save(model_ft.state_dict(), 'bee_classification_resnet.pth')


Classes: ['성충_날개불구바이러스감염증', '성충_응애', '성충_정상', '유충_부저병', '유충_석고병', '유충_응애', '유충_정상']
Dataset size: 329


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can als

Epoch 0/24
----------
train Loss: 1.9261 Acc: 0.3042
val Loss: 1.4200 Acc: 0.5455
Epoch 1/24
----------
train Loss: 1.0328 Acc: 0.7186
val Loss: 0.8050 Acc: 0.7727
Epoch 2/24
----------
train Loss: 0.6229 Acc: 0.8099
val Loss: 0.5720 Acc: 0.8030
Epoch 3/24
----------
train Loss: 0.5127 Acc: 0.8441
val Loss: 0.5043 Acc: 0.8485
Epoch 4/24
----------
train Loss: 0.4038 Acc: 0.8593
val Loss: 0.5493 Acc: 0.8182
Epoch 5/24
----------
train Loss: 0.3406 Acc: 0.8897
val Loss: 0.4076 Acc: 0.8939
Epoch 6/24
----------
train Loss: 0.2862 Acc: 0.9278
val Loss: 0.3748 Acc: 0.9091
Epoch 7/24
----------
train Loss: 0.2306 Acc: 0.9392
val Loss: 0.3633 Acc: 0.8939
Epoch 8/24
----------
train Loss: 0.2407 Acc: 0.9240
val Loss: 0.3803 Acc: 0.8788
Epoch 9/24
----------
train Loss: 0.2386 Acc: 0.9506
val Loss: 0.3493 Acc: 0.8939
Epoch 10/24
----------
train Loss: 0.2671 Acc: 0.9392
val Loss: 0.3493 Acc: 0.8788
Epoch 11/24
----------
train Loss: 0.2285 Acc: 0.9544
val Loss: 0.3682 Acc: 0.8788
Epoch 12/24
--

## inference / testing

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# 학습된 모델 불러오기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)  # 사전 학습 없이 모델 구조만 불러옴

# 모델의 출력 레이어를 학습한 클래스 수에 맞게 설정
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 7)  # 클래스 수에 맞게 변경 (여기서는 7)

# 학습된 가중치 불러오기
# model_path = '/content/drive/MyDrive/제4회 AI교육 해커톤/bee_classification_resnet.pth'
model_path = 'input model path'
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()  # 추론 모드로 전환 (학습용이 아닌 추론용)

# 이미지 전처리 설정 (학습 때와 동일한 전처리 적용)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 추론할 이미지 불러오기
# img_path = '/content/drive/MyDrive/제4회 AI교육 해커톤/Sample/01.원천데이터/data/유충_부저병/002/C_002_001_20230819131224_001_001_000_003.jpg'
img_path = 'input image path'
input_image = Image.open(img_path)

# 이미지 전처리 적용
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # 배치 차원을 추가

# 이미지를 장치에 할당 (GPU 또는 CPU)
input_batch = input_batch.to(device)

# 추론 실행
with torch.no_grad():  # 그래디언트를 계산하지 않음
    output = model(input_batch)

# 소프트맥스 적용하여 확률로 변환
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# 클래스 이름 정의 (학습 시 사용한 클래스 순서와 동일해야 함)
class_names = ['성충_날개불구바이러스감염증',
               '성충_응애',
               '성충_정상',
               '유충_부저병',
               '유충_석고병',
               '유충_응애',
               '유충_정상']

# 가장 높은 확률의 클래스 예측
_, predicted_class = torch.max(probabilities, 0)

# 결과 출력
print(f'Predicted class: {class_names[predicted_class]}')
print(f'Class probabilities: {probabilities}')


Predicted class: 유충_부저병
Class probabilities: tensor([0.0331, 0.0319, 0.0037, 0.9221, 0.0017, 0.0041, 0.0035],
       device='cuda:0')


<ipython-input-12-8c90e91f0ad4>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/제4회 AI교육 해커톤/bee_classifi